# Instalando Bibliotecas

In [ ]:
%pip install torch matplotlib

# Importando bibliotecas

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np

# --- 1. Definição do Modelo Perceptron ---

In [ ]:

class MLP(nn.Module):
    def __init__(self, hidden_layers=[50, 50]):
        super().__init__()
        # Cria as camadas com base na lista de hidden_layers
        layers = []
        input_size = 1
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(nn.ReLU())
            input_size = hidden_size
        layers.append(nn.Linear(input_size, 1))
        
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# --- 2. Geração de Dados ---

In [ ]:

def generate_data(func, start, end, num_points=1000):
    # Gera tensores de entrada (x) e saída (y) para uma dada função.
    X = torch.linspace(start, end, num_points).unsqueeze(1)
    y = func(X)
    return X, y

# --- 3. Função de Treinamento e Validação ---

In [ ]:

def train_and_evaluate(model, train_loader, val_loader, epochs=5000, lr=0.001):
    # Treina o modelo e registra o erro de treinamento e validação.
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        # Fase de Treinamento
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        train_losses.append(running_loss / len(train_loader.dataset))

        # Fase de Validação
        model.eval()
        running_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                running_loss += loss.item() * inputs.size(0)
        val_losses.append(running_loss / len(val_loader.dataset))

        if (epoch + 1) % 500 == 0:
            print(f'Época [{epoch+1}/{epochs}], Perda de Treino: {train_losses[-1]:.8f}, Perda de Validação: {val_losses[-1]:.8f}')
    
    return train_losses, val_losses

# --- 4. Visualização dos Resultados ---

In [ ]:

def plot_results(X_all, y_true, y_pred, train_losses, val_losses, title):
    """Plota os gráficos de função real vs. aproximada e curvas de erro."""
    
    # Converte tensores para numpy para plotagem
    X_all_np = X_all.numpy()
    y_true_np = y_true.numpy()
    y_pred_np = y_pred.numpy()
    
    # Gráfico 1: Função Real vs. Aproximada
    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    plt.plot(X_all_np, y_true_np, label='Função Real', color='blue')
    plt.plot(X_all_np, y_pred_np, '--', label='Função Aproximada', color='red')
    plt.title(f'Aproximação da Função: {title}')
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend()
    plt.grid(True)

    # Gráfico 2: Curvas de Erro
    plt.subplot(1, 2, 2)
    plt.plot(train_losses, label='Erro de Treinamento', color='green')
    plt.plot(val_losses, label='Erro de Validação', color='orange')
    plt.title('Curva de Erro durante o Treinamento')
    plt.xlabel('Época')
    plt.ylabel('Erro (MSE)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# Configurações
NUM_SAMPLES = 1000
VAL_SPLIT = 0.2

# --- Aproximação para f(x) = sin(2x) + cos(3x) ---

In [ ]:
print("--- Aproximando f(x) = sin(2x) + cos(3x) ---")

# Define a função e gera os dados
func_a = lambda x: torch.sin(2 * x) + torch.cos(3 * x)
X_a, y_a = generate_data(func_a, 0, 5, NUM_SAMPLES)

# Divide os dados
dataset_a = TensorDataset(X_a, y_a)
train_size_a = int((1 - VAL_SPLIT) * len(dataset_a))
val_size_a = len(dataset_a) - train_size_a
train_dataset_a, val_dataset_a = random_split(dataset_a, [train_size_a, val_size_a])

train_loader_a = DataLoader(train_dataset_a, batch_size=32, shuffle=True)
val_loader_a = DataLoader(val_dataset_a, batch_size=32)

# Treina
model_a = MLP(hidden_layers=[100, 50])
train_losses_a, val_losses_a = train_and_evaluate(model_a, train_loader_a, val_loader_a)

# Previsão final e plotagem
model_a.eval()
with torch.no_grad():
    y_pred_a = model_a(X_a)
plot_results(X_a, y_a, y_pred_a, train_losses_a, val_losses_a, 'sin(2x) + cos(3x)')

# --- Aproximação para f(x) = 10x^5 + 5x^4 + 2x^3 - 0.5x^2 + 3x + 2 ---

In [ ]:
print("--- Aproximando f(x) = 10x^5 + 5x^4 + 2x^3 - 0.5x^2 + 3x + 2 ---")

# Define a função e gera os dados
func_b = lambda x: 10*x**5 + 5*x**4 + 2*x**3 - 0.5*x**2 + 3*x + 2
X_b, y_b = generate_data(func_b, 0, 5, NUM_SAMPLES)

# Divide os dados
dataset_b = TensorDataset(X_b, y_b)
train_size_b = int((1 - VAL_SPLIT) * len(dataset_b))
val_size_b = len(dataset_b) - train_size_b
train_dataset_b, val_dataset_b = random_split(dataset_b, [train_size_b, val_size_b])

train_loader_b = DataLoader(train_dataset_b, batch_size=32, shuffle=True)
val_loader_b = DataLoader(val_dataset_b, batch_size=32)

# Treina
model_b = MLP(hidden_layers=[200, 100, 50]) # Arquitetura mais robusta para um polinômio de maior grau
train_losses_b, val_losses_b = train_and_evaluate(model_b, train_loader_b, val_loader_b, epochs=10000, lr=0.0005)

# Previsão final e plotagem
model_b.eval()
with torch.no_grad():
    y_pred_b = model_b(X_b)
plot_results(X_b, y_b, y_pred_b, train_losses_b, val_losses_b, 'Polinômio de Grau 5')